# Data analysis and visualization

Code for analysis / visualization


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pylab as plt

df = pd.read_csv('../data/inCites/dataframe.csv', index_col=0)
cgab = pd.read_csv('../data/inCites/countries_groupAB.csv', index_col=0)
countries = list(cgab.index)
#df_c = df[countries]

df['PercOAItems'] = df.OAItems / df.TotItems
df['PercNotOAItems'] = df.NotOAItems / df.TotItems
df['CitR'] = df.CitPerItem_OA/ df.CitPerItem_NotOA


In [ ]:
#df

In [ ]:
sns.scatterplot(data=df, x="PercOAItems", y="CitPerItem_OA", hue="Discipline", palette="deep")

In [ ]:
df.loc[df['CitPerItem_OA'] > 30]

In [ ]:
sns.scatterplot(data=df, x="PercNotOAItems", y="CitPerItem_NotOA", hue="Discipline", palette="deep")

In [ ]:
# Does the N. of OA option trend with a higher citation rate OA/notOA ?
sns.scatterplot(data=df, x="PercOAItems", y="CitR", hue="Discipline", palette="deep")
avg_fullOA = df[df.NotOAItems < 1].groupby('Discipline').CitPerItem_OA.mean()
avg_notOA = df[df.OAItems < 1].groupby('Discipline').CitPerItem_NotOA.mean()


In [ ]:
pd.concat([avg_fullOA, avg_notOA], axis=1).reset_index().rename(columns={'CitPerItem_OA': 'Full OA', 'CitPerItem_NotOA': 'No OA'})

In [ ]:
df[df.OAItems < 1]

In [ ]:
df_c = df[countries+['Discipline']].set_index('Discipline', append=True)
df_c.index.rename(['Journal', 'Discipline'], inplace=True)
df_c = df_c.reorder_levels(['Discipline', 'Journal']).T
#g = sns.scatterplot(data=df_c)
#g.set_xticklabels(g.get_xticklabels(), rotation=30)
sns.set(rc={'figure.figsize':(20,5)})
sns.scatterplot(data=df_c)
t = plt.xticks(rotation=90)



# Tot. N of authors per discipline
#df_c.groupby('Discipline', axis=1).sum()
# Country percentage of authors per journal
#df_c / df_c.sum()

In [ ]:
tdf = df_c / df_c.sum()
sns.scatterplot(data=tdf)
t = plt.xticks(rotation=90)

In [ ]:
df_c.sum()